### Preparing the data

Name of the code:               MT_03_Preparing_the_data

Date of creating file:          07.11.2023

Author:                         Marcin Stasiak

Tables created and saved 
in the code:                    dataframe_v03

Descriction: 

The aim of the code is to:

    - Sort the previous data set, 
    - Create flags based on days past due: 
        - x+_delay_yM flags indicating whether at least one event of x days of delay 
            has occurred within y months previously;
        - x_delay_ever flags indicating if there has ever been an x days of delay 
            event before; 
        - 90_12M flag, potential target variable stating whether there has been at least 
            one 90-day delay event in the next 12M 

Import of libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
import time
import os

Set the option to display all columns and 10 rows

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

File path

In [3]:
file_path = "C:\\Users\\Marcin.Stasiak\\Desktop\\studia\\Praca magisterska\\Dane\\dataframe_v02.pkl"

Load data from the .pkl file

In [4]:
dataframe_v02 = pd.read_pickle(file_path)

Display the data

In [5]:
dataframe_v02

,ID,Acq_Date,Note_Date,Maturity_Date_at_Acq,Loan_Acq_UPB,Amortization_Type,Fixed_Interest,Loan_Product_Type,Original_UPB,Amortization_Term,Interest_Rate,Lien_Position,Transaction ID,Issue_Date,Loan_Acq_LTV,Underwritten_DSCR,Underwritten_DSCR_Type,Original_Term,Original_Interest_Only_Term,Interest_Only_End_Date,Loan_Ever_60_Days_Del,Loss_Sharing_Type,Loss_Sharing_Prc,Number_Properties_Acq,Property_Acq_Total_Unit,Specific_Property_Type,Year_Built,Property_City,Property_State,Property_Zip_Code,Metropolitan_Area,Physical_Occupancy,Liq_Prepay_Code,Liq_Prepay_Date,Foreclosure_Date,Credit_Event_Date,Foreclosure_Value,Lifetime_Net_Credit_Loss_Amount,Sale_Price,Default_Amount,Credit_Event_REO,Reporting_Period_Date,Active_Property,Note_Rate,Current_Maturity_Date,Current_UPB,Delinquency_UPB,Payment_Status,SDQ_Indicator,Modification_Date,Modification,Defeasance_Date,Prepayment_Provision,Prepayment Provision End Date,Affordable_Housing_Type,MCIRT_Deal,MCAS_Deal,DUS_Prepayment_Outcomes,DUS_Prepayment_Segments,Loan_Age,Green_Bond,Social_Bond
0,140296,2000-10-31,1985-07-16,2001-08-10,82501.71,NaN,0,DUS,82501.71,999,5.196,First,NaN,NaT,50.0,NaN,NaN,360,999,NaT,1,NaN,NaN,1,4,Multifamily,999,SAN DIEGO,CA,92102,"SAN DIEGO-CHULA VISTA-CARLSBAD, CA METROPOLITA...",0.0,FullyPaidPrepaid,2000-09-10,NaT,NaT,NaN,NaN,NaN,NaN,999,2001-01-01,1,5.61,2001-08-10,82501.71,NaN,90+,1,NaT,0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999,999,999
1,140296,2000-10-31,1985-07-16,2001-08-10,82501.71,NaN,0,DUS,82501.71,999,5.196,First,NaN,NaT,50.0,NaN,NaN,360,999,NaT,1,NaN,NaN,1,4,Multifamily,999,SAN DIEGO,CA,92102,"SAN DIEGO-CHULA VISTA-CARLSBAD, CA METROPOLITA...",0.0,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,999,2000-11-01,1,NaN,2001-08-10,82501.71,NaN,Current,0,NaT,0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999,999,999
2,140296,2000-10-31,1985-07-16,2001-08-10,82501.71,NaN,0,DUS,82501.71,999,5.196,First,NaN,NaT,50.0,NaN,NaN,360,999,NaT,1,NaN,NaN,1,4,Multifamily,999,SAN DIEGO,CA,92102,"SAN DIEGO-CHULA VISTA-CARLSBAD, CA METROPOLITA...",0.0,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,999,2000-12-01,1,NaN,2001-08-10,82501.71,NaN,Current,0,NaT,0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999,999,999
3,140296,2000-10-31,1985-07-16,2001-08-10,82501.71,NaN,0,DUS,82501.71,999,5.196,First,NaN,NaT,50.0,NaN,NaN,360,999,NaT,1,NaN,NaN,1,4,Multifamily,999,SAN DIEGO,CA,92102,"SAN DIEGO-CHULA VISTA-CARLSBAD, CA METROPOLITA...",0.0,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,999,2000-10-01,1,NaN,2001-08-10,82501.71,NaN,Current,0,NaT,0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999,999,999
4,140297,2000-10-31,1985-07-18,2001-08-10,548872.98,NaN,0,DUS,548872.98,999,5.196,First,NaN,NaT,50.0,NaN,NaN,360,999,NaT,1,NaN,NaN,1,2,Multifamily,999,SAN DIEGO,CA,92116,"SAN DIEGO-CHULA VISTA-CARLSBAD, CA METROPOLITA...",0.0,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,999,2000-11-01,1,NaN,2000-08-10,205598.07,NaN,Current,0,NaT,0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,999,999,999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4467302,8300008176,2021-02-25,2021-02-25,2051-02-01,16400000.00,IO/A/B,1,Non_DUS,16400000.00,480,4.130,First,NaN,NaT,62.4,1.18,UW DSCR NCF,360,24,2023-02-01,0,Pari_Passu,NaN,1,102,Multifamily,1925,NEW YORK,NY,10027,"NEW YORK-NEWARK-JERSEY CITY, NY-NJ-PA METROPOL...",95.0,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,999,2023-01-01,1,4.13,2051-02-01,16400000.00,NaN,Current,0,NaT,0,NaT,"5%(84), 4%(24), 3%(24), 2%(24), 1%(24), O*(180)",NaN,PBHAP_S8,NaN,NaN,NaN,NaN,999,999,999
4467303,8300008176,2021-02-25,2021-02-25,2051-02-01,16400000.00,IO/A/B,1,Non_DUS,16400000.00,480,4.130,First,NaN,NaT,62.4,1.18,UW DSCR NCF,360,24,2023-02-01,0,Pari_Passu,NaN,1,102,Multifamily,1925,NEW YORK,NY,10027,"NEW YORK-NEWARK-JERSEY CITY, NY-NJ-PA METROPOL...",95.0,NaN,NaT,NaT,NaT,NaN,NaN,NaN,NaN,999,2021-06-01,1,4.13,2051-02-01,16400000.00,NaN,Current,0,NaT,0,NaT,"5%(84), 4%(24), 3%(24), 2%(24), 1%(24), O*

Sort the DataFrame by 'ID' and 'Reporting_Period_Date'

In [6]:
sorted_dataframe_v02 = dataframe_v02.sort_values(by=['ID', 'Reporting_Period_Date'])

Resetting the index

In [7]:
sorted_dataframe_v02.reset_index(drop=True, inplace=True)

Creating x+_delay_yM flags indicating whether at least one event of x days of delay 
has occurred within y months previously

(Average runtime of the code: 20 min)

In [9]:
start = time.time()

def process_group(group):
    group = group.copy()

    time_frames = [3, 6, 12, 120]  # The different time frames to check for delays
    delay_types = ['30+', '60+', '90+']  # The different types of delays to check for

    # Create new columns and initialize with 0
    for delay in delay_types:
        for time_frame in time_frames:
            col_name = f'{delay}_delay_{time_frame}M' if time_frame != 120 else f'{delay}_delay_10Y'
            group[col_name] = 0

    for delay, delay_type in zip(delay_types, [30, 60, 90]):
        # Create a binary column for each delay type
        binary_col = group['Payment_Status'].str.contains(delay).astype(int)

        for time_frame in time_frames:
            # Calculate the rolling sum for the binary column
            rolling_sum = binary_col.shift().rolling(window=time_frame, min_periods=1).sum()

            # If the rolling sum is greater than 0, then the delay has occurred within the time frame
            col_name = f'{delay}_delay_{time_frame}M' if time_frame != 120 else f'{delay}_delay_10Y'
            group[col_name] = (rolling_sum > 0).astype(int)

            # Assign 999 to rows that don't have enough previous rows for the specific client
            not_enough_history = group.reset_index().index < time_frame
            group.loc[not_enough_history, col_name] = 999

    return group

# Assuming sorted_dataframe_v02 is defined and contains your data
result_df = sorted_dataframe_v02.groupby('ID').apply(process_group)

# Reset the index
result_df.reset_index(drop=True, inplace=True)

end = time.time()
print("Time elapsed: ", end - start)

Time elapsed:  1647.1050231456757


Creating x_delay_ever flags indicating if there has ever been an x days of delay event before 

(Average runtime of the code: 5 min)

In [11]:
start = time.time()

def process_group_ever(group):
    group = group.copy()

    ever_columns = ['30_delay_ever', '60_delay_ever', '90_delay_ever']
    status_checks = ['30+', '60+', '90+']
    
    for col, status in zip(ever_columns, status_checks):
        # Checking whether a given delay status has occurred in previous lines
        group[col] = group['Payment_Status'].eq(status).cumsum().shift(fill_value=0)
        group[col] = group[col].clip(upper=1)  # Conversion of values greater than 1 into 1
    
    return group

# Apply the function to each group
result_df_ever = result_df.groupby('ID').apply(process_group_ever)

# Reset the index
result_df_ever.reset_index(drop=True, inplace=True)

end = time.time()

print('Time:', end - start)

Time: 341.696573972702


Creating 90_12M flag, potential target variable stating whether there has been at least one 90-day delay event in the next 12M 

(Average runtime of the code: 2s)

In [12]:
start = time.time()

# Creation of a column that has a value of 1 when 'Payment_Status' is '90+', otherwise 0
result_df_ever['is_90'] = (result_df_ever['Payment_Status'] == '90+').astype(int)

# Determining the flag value for 12 rows before the first occurrence of '90+' within 12 rows
result_df_ever['90_12M'] = result_df_ever['is_90'].rolling(window=13, min_periods=1).max().shift(-12).fillna(0).astype(int)

# Deletion of the temporary 'is_90' column
result_df_ever.drop(columns=['is_90'], inplace=True)

# Reset the index
result_df_ever.reset_index(drop=True, inplace=True)

end = time.time()
print(end - start)

1.6511168479919434


Set the default display option

In [13]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Displaying results

In [ ]:
result_df_ever.head(250)

In [ ]:
result_df_ever

Manual checks

In [19]:
desired_ids = [140296, 1673867584, 1673867585, 1673887828]
filtered_df = result_df_ever.loc[result_df_ever['ID'].isin(desired_ids), [
    'ID', 'Reporting_Period_Date', 'Payment_Status', 
    '30+_delay_3M', '30+_delay_6M', '30+_delay_12M', '30+_delay_10Y',
    '60+_delay_3M', '60+_delay_6M', '60+_delay_12M', '60+_delay_10Y',
    '90+_delay_3M', '90+_delay_6M', '90+_delay_12M', '90+_delay_10Y',
    '30_delay_ever', '60_delay_ever', '90_delay_ever',
    '90_12M'
]]
print(filtered_df)


             ID Reporting_Period_Date Payment_Status  30+_delay_3M  \
0        140296            2000-10-01        Current           999   
1        140296            2000-11-01        Current           999   
2        140296            2000-12-01        Current           999   
3        140296            2001-01-01            90+             0   
8    1673867584            2000-01-01        Current           999   
9    1673867584            2000-02-01        Current           999   
10   1673867584            2000-03-01        Current           999   
11   1673867584            2000-04-01        Current             0   
12   1673867584            2000-05-01        Current             0   
13   1673867584            2000-06-01        Current             0   
14   1673867584            2000-07-01        Current             0   
15   1673867584            2000-08-01        Current             0   
16   1673867584            2000-09-01        Current             0   
17   1673867584     

Results:
 - 140296 - client who was sold after 90+
 - 1673867584 - client without delay
 - 1673867585 - client with 30+ and 60+
 - 1673887828 - client with 10Y history and 30+, 60+, and 90+

Specify the path where the pickle file will be saved

In [16]:
pickle_file_path = "C:\\Users\\Marcin.Stasiak\\Desktop\\studia\\Praca magisterska\\Dane\\dataframe_v03.pkl"

Check if the file exists, if it does, delete it

In [17]:
if os.path.exists(pickle_file_path):
    os.remove(pickle_file_path)
    print(f"Existing file at {pickle_file_path} has been deleted.")

Save DataFrame to pickle (4388655 rows, 78 columns)

In [18]:
result_df_ever.to_pickle(pickle_file_path)

print(f"Data has been saved to {pickle_file_path}")

Data has been saved to C:\Users\Marcin.Stasiak\Desktop\studia\Praca magisterska\Dane\dataframe_v03.pkl
